In [1]:
import datetime
import itertools
import pandas as pd
from san.extras.strategy import Strategy

/home/alex/santiment/sanpy/san/__init__.py:31: UserWarning: The package sanpy is out of date. Your version is 0.8.5, the latest is 0.8.11.
  warn(warning)


# Init Strategy

In [2]:
index = Strategy(start_dt='2021-01-01')

# Adding/removing assets

In [3]:
# add reserve asset
index.add_assets({'dai': ['2021-01-01', '2021-01-04']}, assets_type='r')
index.reserve_assets

,asset
index,
2021-01-01,dai
2021-01-02,dai
2021-01-03,dai
2021-01-04,dai


In [4]:
# add non-reserve assets
index.add_assets({
    'ethereum': ['2021-01-01', '2021-01-02', '2021-01-04', '2021-01-04'],
    'uniswap': ['2021-01-01', '2021-01-04']
})
index.assets

,asset
index,
2021-01-01,ethereum
2021-01-01,uniswap
2021-01-02,ethereum
2021-01-02,uniswap
2021-01-03,uniswap
2021-01-04,ethereum
2021-01-04,uniswap


In [5]:
# Remove uniswap for 2021-01-03 - 2021-01-05
index.remove_assets({'uniswap': ['2021-01-03', '2021-01-04']})
index.assets

,asset
index,
2021-01-01,ethereum
2021-01-01,uniswap
2021-01-02,ethereum
2021-01-02,uniswap
2021-01-04,ethereum


# Adding/removing signals

In [6]:
buy_signals = pd.DataFrame({
    'dt': ['2021-01-05', '2021-01-05', '2021-02-10', '2021-03-10'],
    'asset': ['ethereum', 'uniswap', 'ethereum', 'uniswap']
})

buy_signals['trade_percantage'] = buy_signals.apply(lambda x: 0.5 if x['asset'] == 'ethereum' else 0.8, axis=1)
buy_signals['decision_delay'] = datetime.timedelta(days=2)

sell_signals_1 = pd.DataFrame({
    'dt': ['2021-02-05', '2021-03-15'],
    'asset': ['ethereum', 'uniswap']
})

sell_signals_2 = pd.DataFrame({
    'dt': ['2021-03-01'],
    'asset': ['uniswap']
})

In [7]:
# Add buy signals to the index
index.add_signals('b', buy_signals, signal_name='buy')
index.buy_signals

,dt,signal,asset,trade_percantage,decision_delay
2021-01-07,2021-01-05,buy,ethereum,0.5,2 days
2021-01-07,2021-01-05,buy,uniswap,0.8,2 days
2021-02-12,2021-02-10,buy,ethereum,0.5,2 days
2021-03-12,2021-03-10,buy,uniswap,0.8,2 days


In [8]:
# Add sell signals to the index
index.add_signals('s', sell_signals_1, signal_name='sell_1')
index.add_signals('s', sell_signals_2, signal_name='sell_2')
index.sell_signals

,dt,signal,asset,trade_percantage,decision_delay
2021-02-05,2021-02-05,sell_1,ethereum,NaN,0 days
2021-03-15,2021-03-15,sell_1,uniswap,NaN,0 days
2021-03-01,2021-03-01,sell_2,uniswap,NaN,0 days


In [9]:
# Remove part of the sell signals
index.remove_signals('s', signal_name='sell_2')
index.sell_signals

,dt,signal,asset,trade_percantage,decision_delay
2021-02-05,2021-02-05,sell_1,ethereum,NaN,0 days
2021-03-15,2021-03-15,sell_1,uniswap,NaN,0 days


In [10]:
# Define rebalance signals df
rebalance_signals = pd.DataFrame({
    'dt': ['2021-02-01', '2021-03-01']
})

In [11]:
# Add rebalance signals
index.add_signals('r', rebalance_signals, signal_name='rebalance')
index.rebalance_signals

,dt,signal,asset,trade_percantage,decision_delay
2021-02-01,2021-02-01,rebalance,NaN,NaN,0 days
2021-03-01,2021-03-01,rebalance,NaN,NaN,0 days


# Setting prices

In [12]:
prices = pd.DataFrame(
    list(itertools.product(['2021-01-01', '2021-01-02', '2021-01-03'], ['ethereum', 'dai', 'uniswap'])),
    columns=['dt', 'asset']
).set_index('dt')
prices['price'] = [4000, 1, 100, 4100, 1, 90, 4200, 1, 110]

In [13]:
index.set_prices(prices)

In [14]:
index.prices

,asset,price,price_change
dt,,,
2021-01-01,dai,1,1.000000
2021-01-01,ethereum,4000,1.000000
2021-01-01,uniswap,100,1.000000
2021-01-02,dai,1,1.000000
2021-01-02,ethereum,4100,1.025000
2021-01-02,uniswap,90,0.900000
2021-01-03,dai,1,1.000000
2021-01-03,ethereum,4200,1.024390
2021-01-03,uniswap,110,1.222222


# Adding periodic rebalance

In [15]:
# # add default monthly rebalancing
# index.add_periodic_rebalance('0 0 1 * *', skip_rebalance_on_init=True)

# Init

In [16]:
index.build_portfolio('2021-01-01', '2021-01-03')

KeyError: Timestamp('2021-01-01 00:00:00', freq='D')

In [17]:
index.portfolio

,asset,share
dt,,
